# QMF: Final Project

*By Basri, Clara, Daniel, Elisa, José, Karim, Nani*

In [3]:
import pandas as pd

In [15]:
df_covid_vaccines = pd.read_csv("./datasets/covid_vaccines.csv", sep=';', usecols=['reg', 'jour', 'n_cum_dose1', 'n_cum_complet', 'n_cum_rappel'], parse_dates=['jour'])
df_covid_vaccines.rename(columns={'reg': 'region', 'jour': 'date', 'n_cum_dose1': 'n_first_dose', 'n_cum_complet': 'n_second_dose', 'n_cum_rappel': 'n_third_dose'}, inplace=True)
df_covid_vaccines = df_covid_vaccines.groupby(by='date').sum().drop(columns=['region'])

In [11]:
df_covid_vaccines

,n_first_dose,n_second_dose,n_third_dose
date,,,
2020-12-27,273,34,0
2020-12-28,587,56,0
2020-12-29,791,65,0
2020-12-30,1065,76,0
2020-12-31,1264,89,0
...,...,...,...
2021-11-10,51419029,50309451,4145106
2021-11-11,51422118,50314820,4158161
2021-11-12,51449186,50356367,4351280


In [22]:
df_covid_vaccines = pd.read_csv(
    "https://static.data.gouv.fr/resources/donnees-relatives-aux-personnes-vaccinees-contre-la-covid-19-1/20211203-190546/vacsi12-fra-2021-12-03-19h05.csv", 
    sep=';', 
    names=['fra', 'date', '_', '__', '___', 'n_first_dose', 'n_second_dose', 'n_third_dose', '____', '_____', '______'], 
    usecols=['fra', 'date', 'n_first_dose', 'n_second_dose', 'n_third_dose'], 
    skiprows=1, 
    parse_dates=['date']
)

In [33]:
df_covid_indicators = pd.read_csv(
    "https://static.data.gouv.fr/resources/synthese-des-indicateurs-de-suivi-de-lepidemie-covid-19/20211205-190939/table-indicateurs-open-data-france-2021-12-05-19h09.csv", 
    usecols=['date', 'dc_tot', 'conf_j1', 'TO', 'R'],  
    parse_dates=['date']
)
df_covid_indicators.rename(columns={'date': 'date', 'dc_tot': 'n_deaths', 'conf_j1': 'n_new_cases', 'TO': 'hosp_occupation_rate', 'R': 'virus_reproduction_factor'}, inplace=True)
df_covid_indicators.fillna(0, inplace=True)

In [34]:
df_covid_indicators = pd.read_csv(
    "https://www.insee.fr/fr/information/3128533?INDICATEUR=2238611%2B2238612", 
    # usecols=['date', 'dc_tot', 'conf_j1', 'TO', 'R'],  
    # parse_dates=['date']
)
# df_covid_indicators.rename(columns={'date': 'date', 'dc_tot': 'n_deaths', 'conf_j1': 'n_new_cases', 'TO': 'hosp_occupation_rate', 'R': 'virus_reproduction_factor'}, inplace=True)
# df_covid_indicators.fillna(0, inplace=True)
df_covid_indicators

ParserError: Error tokenizing data. C error: Expected 1 fields in line 12, saw 3
